## Цель исследования.

### Проверка трёх гипотез: 
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

### Ход исследования.
Качество данных в предоставленном файле (yandex_music_project.csv) неизвестно, потому предварительно был произведён обзор данных и исправлены ошибки. Затем проведено исследование, которое позволило сделать вывод относительно гипотез.

### 1. Обзор данных.

In [4]:
import pandas as pd

df = pd.read_csv('C:\pyprojects\DA_Yandex\yandex_music_project.csv')

df.info()
df.head(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


В датафрейме представлены данные, описывающие как сам трек (название композиции, имя исполнителя, жанр), так и пользователя (айди, город, день и время начала прослушивания). Кроме того, из полученной информации можно сделать вывод, что в таблице есть пропущенные значения и проблема с именованием столбцов. 

Первым делом займёмся предобработкой данных.

1. Переименум столбцы

In [5]:
df.columns # проверка названий

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
df = df.rename(columns = {'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'}) # замена названий

In [7]:
df.columns # проверка после замены

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

2. Обработаем пропуски. 

In [8]:
df.isna().sum() # проверим число пропусков

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски в названиях и авторах совершенно не критичны и не повлияют на результат исследования. Пропуски в жанрах представляют проблему. Её масштаб будет ясен в процессе. 
Заменим пропуски на значение 'unknown'.

In [9]:
# перебор названий столбцов в цикле и замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Убедимся, что пропуски ликвидированы.

In [10]:
df.isna().sum() # проверим число пропусков

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Следом займёмся дубликатами. Во-первых, посчитаем явные дубли и удалим им.

In [11]:
df.duplicated().sum() # число дубликатов

3826

In [12]:
# удаление явных дубликатов
df = df.drop_duplicates().reset_index(drop=True)

In [13]:
df.duplicated().sum() # число дубликатов

0

Теперь найдём неявные дубликаты и устраним их. 

In [14]:
# Просмотр уникальных значений в колонке жанров
genre = df['genre']
genre.sort_values().unique()


array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Неявные дубли есть для жанров hiphop и electronic: hip, hip-hop, hop и электроника. Заменим их.


In [15]:
df['genre'] = df['genre'].replace('hip', 'hiphop')
df['genre'] = df['genre'].replace('hop', 'hiphop')
df['genre'] = df['genre'].replace('hip-hop', 'hiphop')
df['genre'] = df['genre'].replace('электроника', 'electronic')

In [16]:
genre = df['genre']
genre.sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

## Проверка гипотез

### 1. Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это, для чего:

• разделим пользователей Москвы и Санкт-Петербурга; 

• сравним, сколько треков послушала каждая группа ользователей в понедельник, среду и пятницу.

In [18]:
# Сначала все данные выведем отдельно: прослушивание по городам и прослушивание по дням.
# Прослушивание по городам
listening = df.groupby('city')['user_id'].count()
print(listening)

# Прослушивание по дням
listening_day = df.groupby('day')['genre'].count()
print(listening_day)

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64
day
Friday       21840
Monday       21354
Wednesday    18059
Name: genre, dtype: int64


Объединим оба расчёта в одну функцию. Она имеет два параметра (day, city) и считает прослушивания для заданных дня и города. В функции в переменную сохраним строки, в которых в колонке day значение равно параметну day, а в колонке city - параметру city. Затем считаем значения в user_id получившейся таблицы, которые сохраняем в переменной и возвращаем в функции.


In [20]:
# Объявляется функция с двумя параметрами: day, city.
# В переменной track_list сохраняются те строки таблицы df, для которых 
# значение в столбце 'day' равно параметру day и одновременно значение
# в столбце 'city' равно параметру city

def number_tracks(day, city):
    
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    
    track_list_count = track_list['user_id'].count()
    
    return track_list_count

# В переменной track_list_count сохраняется число значений столбца 'user_id',
# рассчитанное методом count() для таблицы track_list.
# Функция возвращает число - значение track_list_count.
    
# Функция для подсчёта прослушиваний для конкретного города и дня.
# С помощью последовательной фильтрации с логической индексацией она 
# сначала получит из исходной таблицы строки с нужным днём,
# затем из результата отфильтрует строки с нужным городом,
# методом count() посчитает количество значений в колонке user_id. 
# Это количество функция вернёт в качестве результата

In [21]:
# количество прослушиваний в Москве по понедельникам
number_tracks('Monday', 'Moscow')

15740

In [22]:
# количество прослушиваний в Санкт-Петербурге по понедельникам
number_tracks('Monday', 'Saint-Petersburg')

5614

In [23]:
# количество прослушиваний в Москве по средам
number_tracks('Wednesday', 'Moscow')

11056

In [24]:
# количество прослушиваний в Санкт-Петербурге по средам
number_tracks('Wednesday', 'Saint-Petersburg')

7003

In [25]:
# количество прослушиваний в Москве по пятницам
number_tracks('Friday', 'Moscow')

15945

In [26]:
# количество прослушиваний в Санкт-Петербурге по пятницам
number_tracks('Friday', 'Saint-Petersburg')

5895

Для удобства чтения создадим датафрейм.

In [27]:
data = [['Moscow', 15740, 11056, 15945], ['Saint-Petersburg', 5614, 7003, 5895]]
columns_names = ['city', 'monday', 'wednesday', 'friday']
result_table = pd.DataFrame(data = data, columns = columns_names)
display(result_table)

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


## Вывод по первой гипотезе.

Данные показывают разницу поведения пользователей:

1. В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
2.  Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

### Это значит, данные говорят в пользу первой гипотезы.







### 2. Музыка в начале недели и в конце.

Гипотеза предполагает, что в Мск и СПб утром понедельника и вечером пятницы преобладают разные жанры.

Для проверки гипотезы создадим две таблицы (по одной на город) и функцию для расчёта числа прослушанных треков в заданный временной интервал заданного дня.

In [31]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']


In [35]:
# получение таблицы spb_general из тех строк таблицы df,
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']


In [38]:
# Объявление функции genre_weekday() с параметрами table, day, time1, time2,
# которая возвращает информацию о самых популярных жанрах в указанный день в
# заданное время:
def genre_weekday(table, day, time1, time2):
    
# 1) в переменную genre_df сохраняются те строки переданного датафрейма table, для
#    которых одновременно:
#    - значение в столбце day равно значению аргумента day
#    - значение в столбце time больше значения аргумента time1
#    - значение в столбце time меньше значения аргумента time2
#    Используется последовательная фильтрация с помощью логической индексации.

    genre_df = table[(table['day'] == day) & (table['time'] > time1) & (table['time'] < time2)]
    

# 2) сгруппировать датафрейм genre_df по столбцу genre, взять один из его
#    столбцов и посчитать методом count() количество записей для каждого из
#    присутствующих жанров, получившийся Series записать в переменную
#    genre_df_count

    genre_df_count = genre_df.groupby('genre')['genre'].count()

# 3) отсортировать genre_df_count по убыванию встречаемости и сохранить
#    в переменную genre_df_sorted

    genre_df_sorted = genre_df_count.sort_values(ascending = False)
    
# 4) вернуть Series из 10 первых значений genre_df_sorted, это будут топ-10
#    популярных жанров (в указанный день, в заданное время)

    return genre_df_sorted[:10]

In [40]:
# вызов функции для утра понедельника в Москве 
# объекты, хранящие время, являются строками и сравниваются как строки
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [41]:
# вызов функции для утра понедельника в Петербурге
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [43]:
# вызов функции для вечера пятницы в Москве
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [42]:
# вызов функции для вечера пятницы в Петербурге
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

## Вывод по второй гипотезе

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

В Москве и Петербурге слушают **похожую музыку**. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

В Москве **пропущенных значений оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров**. Значит, пропущенные значения занимают существенную долю в данных и **угрожают достоверности** исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:

Пользователи слушают похожую музыку в начале недели и в конце.
Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.
Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

### 3. Предпочтения по жанрам в Мск и СПб

По гипотезе в в СПб слушают чаще всего рэп, а в Мск - жанр поп. 

Создадим таблицы для каждого города и посчитаем методом `count()` число треков, прослушанных в каждом жанре. Для решения будет достаточно 10 позиций.

In [44]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [45]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)

In [48]:
moscow_genres.head(10)


genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [49]:
spb_genres.head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

## Выводы по третьей гипотезе.

Гипотеза **частично** подтвердилась:

Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.


# Итоги исследования

Были проверены три гипотезы. Установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.
Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
в Москве слушают музыку жанра “world”,
в Петербурге — джаз и классику.
Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.
Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.